<a href="https://colab.research.google.com/github/WeimusT/Quantitative_trading/blob/main/Statistical_Factor_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

The Capital Asset Pricing Model (CAPM) is a model to estimate the return of a stock ($E[R]$) using market return ($R_m$), risk free return ($R_f$) and the volatility of the stock ($\beta$). It is defined as 

$E[R]=R_f + \beta(R_m-R_f)$.



Arbitrage Pricing T

Factor model

[Fama French Model](https://www.investopedia.com/terms/f/famaandfrenchthreefactormodel.asp)

In [ ]:
# Calculate last 30 day daily beta of Tenable

In [2]:
import math
import numpy as np
from numpy.linalg import eig, eigh
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import Ridge
import time

In [5]:
# # Enable when using colab editor
# from google.colab import drive
# drive.mount("/content/drive")
# file_path = "/content/drive/MyDrive/data/IJR_20080114.txt"

In [4]:
# Constants
lookback = 252
numFactors = 5
topN = 50

In [6]:
# Load S&P 600 small cap stocks
file_path = "/Users/wtai/Projects/Quantitative_trading/data/IJR_20080131.txt"
df = pd.read_table(file_path)
df["Date"] = df["Date"].astype("int")
df.set_index("Date", inplace=True)
df.fillna(method="ffill", inplace=True)
df.sort_index(inplace=True)
df

,ABCW,ABFS,ABM,ACAT,ACLS,ACO,ACTL,ADPT,AEIS,AGL,...,WTS,WW,WWW,WXS,XRIT,ZEP,ZLC,ZNT,ZQK,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
20040115,22.1609,30.9237,16.4747,26.4522,11.32,23.3373,26.93,10.10,27.430,21.6689,...,22.6328,24.3135,13.8953,NaN,11.8467,NaN,26.115,19.7760,8.865,NaN
20040116,22.2135,30.2284,16.4129,25.7508,12.56,19.5716,27.92,10.50,27.390,21.5655,...,22.2980,23.9795,13.7789,NaN,11.9759,NaN,26.015,19.7346,8.970,NaN
20040120,22.6166,30.6230,16.5188,26.3465,13.22,22.5940,28.00,11.05,27.500,21.1893,...,22.4851,24.2447,13.6109,NaN,12.6120,NaN,26.125,20.9153,8.965,NaN
20040121,22.7393,30.9425,16.4394,26.4128,12.77,20.7356,27.00,10.68,26.780,21.1046,...,22.5540,24.0188,13.6368,NaN,12.7014,NaN,26.200,20.8976,8.935,NaN
20040122,22.6867,31.4311,16.2541,25.6153,12.23,20.0216,27.05,10.18,25.578,20.9259,...,22.6033,24.1858,13.4623,NaN,12.6815,NaN,26.760,20.7441,8.775,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080128,23.4901,28.2800,20.1300,7.8700,4.06,24.6600,11.43,3.02,10.540,17.6500,...,27.2500,47.7700,21.5200,28.80,9.8700,15.77,15.780,43.8201,8.850,17.9000
20080129,23.7500,28.8100,20.2700,7.8700,4.13,24.7400,11.73,2.95,10.730,17.5300,...,28.9400,47.1900,21.8200,28.80,9.8900,15.58,16.180,43.9600,8.780,17.3200
20080130,23.1600,30.0000,19.8655,8.7400,4.00,24.4200,11.73,2.92,10.810,17.0800,...,28.7500,48.1200,23.3000,29.00,9.9488,15.99,15.930,42.1900,9.060,17.9400


In [7]:
# Daily return
dailyret = df.pct_change()
dailyret

,ABCW,ABFS,ABM,ACAT,ACLS,ACO,ACTL,ADPT,AEIS,AGL,...,WTS,WW,WWW,WXS,XRIT,ZEP,ZLC,ZNT,ZQK,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
20040115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20040116,0.002374,-0.022484,-0.003751,-0.026516,0.109541,-0.161360,0.036762,0.039604,-0.001458,-0.004772,...,-0.014793,-0.013737,-0.008377,NaN,0.010906,NaN,-0.003829,-0.002093,0.011844,NaN
20040120,0.018147,0.013054,0.006452,0.023133,0.052548,0.154428,0.002865,0.052381,0.004016,-0.017445,...,0.008391,0.011059,-0.012193,NaN,0.053115,NaN,0.004228,0.059829,-0.000557,NaN
20040121,0.005425,0.010433,-0.004807,0.002516,-0.034039,-0.082252,-0.035714,-0.033484,-0.026182,-0.003997,...,0.003064,-0.009318,0.001903,NaN,0.007088,NaN,0.002871,-0.000846,-0.003346,NaN
20040122,-0.002313,0.015791,-0.011272,-0.030194,-0.042287,-0.034434,0.001852,-0.046816,-0.044884,-0.008467,...,0.002186,0.006953,-0.012796,NaN,-0.001567,NaN,0.021374,-0.007345,-0.017907,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080128,0.062867,0.030237,0.021309,0.026076,0.007444,-0.040467,-0.030534,0.003322,0.021318,0.047478,...,0.001838,0.016167,0.015573,0.005236,-0.032116,0.008312,0.064059,0.031899,0.047337,-0.000045
20080129,0.011064,0.018741,0.006955,0.000000,0.017241,0.003244,0.026247,-0.023179,0.018027,-0.006799,...,0.062018,-0.012142,0.013941,0.000000,0.002026,-0.012048,0.025349,0.003193,-0.007910,-0.032402
20080130,-0.024842,0.041305,-0.019956,0.110546,-0.031477,-0.012935,0.000000,-0.010169,0.007456,-0.025670,...,-0.006565,0.019708,0.067828,0.006944,0.005945,0.026316,-0.015451,-0.040264,0.031891,0.035797


Principal Component Analysis (PCA) is used to identify the pricipal components of stocks' daily returns. The first component usually resembles the return caused by market movement. In this case, the first component should be very similar to the S&P 600 ETF return at the same period.

The factor models regress principal components to daily returns. Suppose there are k assets and T periods. Let $r_{it}$ be the (excess) return of asset $i$ at time $t$. Assuming $r_it$ is stationary.


The parameter indicates how much does the . It resembles the beta.

In this example, the most prominent 5 factors are regressed to the daily returns of each stock. 

In [8]:
positionsTable = np.zeros(df.shape)
end_index = df.shape[0]
for t in tqdm(np.arange(lookback+1, end_index)):
  # Build a model base the lookback period for each day t in the backtest period.
  R = dailyret.iloc[t-lookback+1:t,].T # Get the lookback period return for each day in the test period
  hasData = np.where(R.notna().all(axis=1))[0] # Find stocks with empty values
  R.dropna(inplace=True) # Drop stocks with empty values
  # Principal Component Regression
  pca = PCA() 
  X = pca.fit_transform(R.T)[:, :numFactors]
  X = sm.add_constant(X)
  y1 = R.T # Why regress to all daily return in the lookback period? Should it not be the next day return? No. Check the above text field.
  clf = MultiOutputRegressor(LinearRegression(fit_intercept=False), n_jobs=4).fit(X, y1)
  Rexp = np.sum(clf.predict(X), axis=0) # Total return in the backtest period
  R = dailyret.iloc[t-lookback+1:t+1,].T
  idxSort = Rexp.argsort() # Sort total return in ascending order
  positionsTable[t, hasData[idxSort[np.arange(0, topN)]]] = -1 # Short stocks with negative predicted return
  positionsTable[t, hasData[idxSort[np.arange(-topN, -1)]]] = 1 # Buy stocks with positive predicted return

100%|██████████| 766/766 [03:21<00:00,  3.80it/s]


In [20]:
capital=np.nansum(np.array(abs(pd.DataFrame(positionsTable)).shift()), axis=1)
positionsTable[capital==0, ]=0
capital[capital==0]=1
ret = np.nansum(np.array(pd.DataFrame(positionsTable).shift())*np.array(dailyret), axis=1)/capital
avgret = np.nanmean(ret)*252
avgstdev = np.nanstd(ret)*math.sqrt(252)
Sharpe = avgret / avgstdev
print(avgret)
print(avgstdev)
print(Sharpe)

-0.00013463038245692152
0.07706594576883878
-0.0017469503697618751
